In [1]:
import cv2
from os import walk
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

# import os
import azureml.core
from azureml.core import Workspace, Dataset, Experiment, ScriptRunConfig, Environment
# from azureml.data.datapath import DataPath
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.runconfig import DockerConfiguration
from azureml.core.conda_dependencies import CondaDependencies

In [2]:
def env_packages():
    conda_dep = CondaDependencies()
    conda_dep.add_channel('pytorch')
    conda_dep.add_channel('defaults')
    
    conda_dep.add_conda_package('python=3.8')
    conda_dep.add_conda_package('pip')
    conda_dep.add_conda_package('python-spams')
    
    conda_dep.add_pip_package('wandb>=0.10.30')
    conda_dep.add_pip_package('opencv-python-headless>=4.4.0.46')
    conda_dep.add_pip_package('numpy')
    conda_dep.add_pip_package('matplotlib>=3.3.3')
    conda_dep.add_pip_package('torch>=1.8.1')
    conda_dep.add_pip_package('torch-summary')
    conda_dep.add_pip_package('torchvision==0.5.0')
    conda_dep.add_pip_package('pandas~=1.4.1')
    conda_dep.add_pip_package('azureml-core~=1.39.0')
    conda_dep.add_pip_package('torchvision')
    conda_dep.add_pip_package('tqdm')
    
    return conda_dep

In [3]:
ws = Workspace.from_config('config.json', auth=InteractiveLoginAuthentication())
print(ws.name, ws.location, ws.resource_group, sep='\t')

a0047stufiitml01	westeurope	a0047-STUFIIT-ML01


In [4]:
experiment = Experiment(workspace=ws, name='BCSS-test-run-1')

In [5]:
dataset = Dataset.get_by_name(workspace=ws, name='BCSS-Dataset')

In [6]:
dc = DockerConfiguration(use_docker=True)

config = ScriptRunConfig(source_directory='../',
                         script='./scripts/main.py',
                         arguments=['--wandb', "584287ef3bc4e3311465171c04c9525858e97893",
                                    '--data_path', dataset.as_mount()],
                         compute_target='Slimak-K80',
                         docker_runtime_config=dc)

In [7]:
# azureml.core.runconfig.DockerConfiguration(use_docker=True)
env = Environment('slimak-env')
env.python.conda_dependencies = env_packages()
env.save_to_directory('../', overwrite=True)
env.register(workspace=ws)

# env = Environment.get(workspace=ws, name='Torch1.8-Opencv-wandb')

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220208.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "slimak-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge",
  

In [8]:
config.run_config.environment = env

In [15]:
run = experiment.submit(config)

In [16]:
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/runs/BCSS-test-run-1_1647469296_e0f96b54?wsid=/subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourcegroups/a0047-STUFIIT-ML01/workspaces/a0047stufiitml01&tid=5dbf1add-202a-4b8d-815b-bf0fb024e033


In [1]:
image_dir_path = r'..\\BCSS\\CrowdsourcingDataset-Amgadetal2019\\BCSS-dataset\\train\\images'
mask_dir_path = r'..\\BCSS\\CrowdsourcingDataset-Amgadetal2019\\BCSS-dataset\\train\\masks'
    
images_tiles_dir_train = r'E:\\Desktop\\Bakalarka\\BCSS\\CrowdsourcingDataset-Amgadetal2019\\BCSS-dataset\\debug\\'
images_tiles_dir_test = r'E:\\Desktop\\Bakalarka\\BCSS\\CrowdsourcingDataset-Amgadetal2019\\BCSS-dataset\\debug\\'
masks_tiles_dir_train = r'E:\\Desktop\\Bakalarka\\BCSS\\CrowdsourcingDataset-Amgadetal2019\\BCSS-dataset\\debug\\'
masks_tiles_dir_test = r'E:\\Desktop\\Bakalarka\\BCSS\\CrowdsourcingDataset-Amgadetal2019\\BCSS-dataset\\debug\\'

test_institutes = ['OL', 'LL', 'C8', 'BH', 'AR', 'A7', 'A1']
tile_size = 128 # 128px
shift_size = tile_size - 20 # 108px

testing_dir = r'..\BCSS\CrowdsourcingDataset-Amgadetal2019\testing'

In [2]:
images_names = []
for (dirpath, dirname, filename) in walk(image_dir_path):
    images_names.extend(filename)
    break

NameError: name 'walk' is not defined

In [ ]:
images_names = 'TCGA-A2-A0CM-DX1_xmin18562_ymin56852_MPP-0.2500.png'

**Function for path generation**

In [24]:
def get_path(image_name, row, column, identifier):
    path = ""
    postfix = '[' + str(int(row/shift_size)) + ',' + str(int(column/shift_size)) + ']' + ".png"
    institute = image_name.split('-')[1]
    if institute in test_institutes:
        # Test group
        if identifier == 'M' or identifier == 'm':
            # Mask
            path = masks_tiles_dir_test + '\\' + image_name[0:-4] + postfix
        elif identifier == 'I' or identifier == 'i':
            # Image
            path = images_tiles_dir_test + '\\' + image_name[0:-4] + postfix
        else:
            return 0
    else:
        # Train group
        if identifier == 'M' or identifier == 'm':
            # Mask
            path = masks_tiles_dir_train + '\\' + image_name[0:-4] + postfix
        elif identifier == 'I' or identifier == 'i':
            # Image
            path = images_tiles_dir_train + '\\' + image_name[0:-4] + postfix
        else:
            return 0
    return path

Split images into tiles and save them to files

In [25]:
for number, image_name in tqdm(enumerate(images_names)):
    # Load image/mask from file
    image = cv2.imread(image_dir_path + "\\" + image_name, cv2.IMREAD_COLOR)
    mask = cv2.imread(mask_dir_path + "\\" + image_name, cv2.IMREAD_GRAYSCALE)
    
    # Get image/mask parameters
    image_height, image_width, image_channels = image.shape
    mask_height, mask_width = mask.shape
    
    print(f"IMAGE - height: {image_height}, width: {image_width}, channels: {image_channels}")
    print(f"MASK - height: {mask_height}, width: {mask_width}")
    
    row_index = 0
    column_index = 0
    
    # Cycle by shift and cut images untill it is possible
    while row_index + tile_size <= image_height:
        column_index = 0 
        while column_index + tile_size <= image_width:
            image_tile = image[row_index : row_index + tile_size, column_index : column_index + tile_size]
            mask_tile = mask[row_index : row_index + tile_size, column_index : column_index + tile_size]
            
            cv2.imwrite(get_path(image_name, row_index, column_index, 'I'), image_tile)
            cv2.imwrite(get_path(image_name, row_index, column_index, 'M'), mask_tile)
            
            column_index += shift_size   
        row_index += shift_size
    
    # Add tiles from last column (image width - tile size)
    row_index_tmp = 0 
    last_column = image_width - tile_size
    while row_index_tmp + tile_size <= image_height:
        image_tile = image[row_index_tmp : row_index_tmp + tile_size, last_column : last_column + tile_size]
        mask_tile = mask[row_index_tmp : row_index_tmp + tile_size, last_column : last_column + tile_size]
        
        cv2.imwrite(get_path(image_name, row_index_tmp, column_index, "I"), image_tile)
        cv2.imwrite(get_path(image_name, row_index_tmp, column_index, "M"), mask_tile)
        
        row_index_tmp += shift_size
    
    # Adding tiles from last row (image height - tile size )
    column_index_tmp = 0
    last_row = image_height - tile_size
    while column_index_tmp + tile_size <= image_width:
        image_tile = image[last_row : last_row + tile_size, column_index_tmp : column_index_tmp + tile_size]
        mask_tile = mask[last_row : last_row + tile_size, column_index_tmp : column_index_tmp + tile_size]
    
        cv2.imwrite(get_path(image_name, row_index, column_index_tmp, 'I'), image_tile)
        cv2.imwrite(get_path(image_name, row_index, column_index_tmp, 'M'), mask_tile)
        
        column_index_tmp += shift_size
           
    # Adding one last tile from last column and row
    image_tile = image[last_row : last_row + tile_size, last_column : last_column + tile_size]
    mask_tile = mask[last_row : last_row + tile_size, last_column : last_column + tile_size]
    
    cv2.imwrite(get_path(image_name, row_index, column_index, 'I'), image_tile)
    cv2.imwrite(get_path(image_name, row_index, column_index, 'M'), mask_tile)
    
    print("Image nr.: " + str(number) + ", produced: " + str( int(row_index/shift_size) * int(column_index/shift_size) ) + " tiles")


0it [00:00, ?it/s]

IMAGE - height: 2467, width: 3394, channels: 3
MASK - height: 2467, width: 3394



1it [00:01,  1.79s/it]

Image nr.: 0, produced: 682 tiles
IMAGE - height: 3426, width: 4838, channels: 3
MASK - height: 3426, width: 4838



2it [00:05,  2.68s/it]

...

Image nr.: 148, produced: 1804 tiles
IMAGE - height: 3257, width: 3243, channels: 3
MASK - height: 3257, width: 3243



150it [26:47,  6.89s/it]

Image nr.: 149, produced: 841 tiles
IMAGE - height: 2316, width: 1864, channels: 3
MASK - height: 2316, width: 1864


151it [26:50, 10.66s/it]

Image nr.: 150, produced: 357 tiles
